In [6]:
# Import our packages for numerical computing
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy.linalg as sla
%matplotlib inline

__Compare Classical GS against Modified GS__
- Review these implementations at home, comparing to the pseudocode in Trefethen and Bau


In [7]:
# First, define function for classical Gram-Schmidt
def classical_GS(A):
    '''
    Carry out classical Gram-Schmidt on A
    Return reduced Q R
    '''
    m = A.shape[0]
    n = A.shape[1]
    
    # Preallocate Q and R (more efficient)
    Q = np.zeros( (m,n) )
    R = np.zeros( (n,n) )
    
    # Loop over columns of A
    for j in range(n):
        # Create orthonormal qj based on column j of A
        vj = A[:,j]
        
        # Orthogonalize vj w.r.t. all the previous columns of A
        #  Note: Python will not execute this loop for the i==j case 
        for i in range(j):
            R[i,j] = np.dot(Q[:,i], A[:,j])
            vj = vj - R[i,j]*Q[:,i]
        
        # Set column j in Q
        R[j,j] = sla.norm(vj)
        Q[:,j] = vj/R[j,j]
        
    ## End loops    
    return (Q,R)
    
    

In [8]:
# Next, define function for Modified GS
def mgs(A):
    '''
    Carry out modified Gram-Schmidt on A
    Return reduced Q R
    '''
    m = A.shape[0]
    n = A.shape[1]
    
    # Preallocate Q and R (more efficient)
    Q = np.zeros( (m,n) )
    R = np.zeros( (n,n) )
    
    # Skip pre-assignment loop of
    #    v_i <-- a_i
    # as done in Trefethen and Bau. We will just over-write A
    # as we compute, i.e., v_i will always be the same thing as 
    # a_i. This saves space.
    
    # Loop over columns of A
    for i in range(n):
        # Form and store qj as column j in Q
        R[i,i] = sla.norm(A[:,i])
        Q[:,i] = A[:,i]/R[i,i]
        
        # Orthogonalize all columns j>i of V (really A) w.r.t. qi vj w.r.t. all the previous columns of A
        #  Note: Python will not execute this loop for the i==j case 
        for j in range(i+1,n):
            R[i,j] = np.dot(Q[:,i], A[:,j])
            A[:,j] = A[:,j] - R[i,j]*Q[:,i]
        
    ## End loops    
    return (Q,R)
    


In [9]:
# Sanity check that the implementation is OK
A = sp.rand(5,5)
np.set_printoptions(precision=3, suppress=True)
[Q1,R1] = classical_GS(A)
[Q2,R2] = mod_GS(A)
print(Q1-Q2)
print("----")
print(R1-R2)

NameError: name 'mod_GS' is not defined

__Now let's run an experiment__
- Connect diagonal of $R$ to singular values
- We first create a matrix $A$ with known singular values, using the the library $QR$ routine to generate the $U$ and $V$ for the manufactured SVD.  
- That is, we manufacture the diagonal matrix $\Sigma$, and form $A$ with
$$ A = U \Sigma V^* $$
so that we know the singular values a priori.  
- We generate the $U$ and $V^*$ by using $QR$ on random matrices.

In [ ]:
# Generate this A with the below geometrically shrinking singular values
n = 80
[U,X] = sla.qr( sp.rand(n,n))
[Vt,X] = sla.qr( sp.rand(n,n))
S = sp.diag( 2.0**sp.arange(-1,-81,-1) )
A = np.dot(U, np.dot(S, Vt))
print(sp.diag(S[0:8]))


Now, we generate two separate $QR$ factorizations using both classical and modified G-S
- We plot the diagonal elements of both $R$, followed by a discussion of this experiment below

In [ ]:
[Q1,R1] = classical_GS(A.copy())
[Q2,R2] = mod_GS(A.copy())
# Plot diagonal elements of R1 and R2
plt.semilogy(sp.diag(R1), 'sk', linewidth=2)
plt.semilogy(sp.diag(R2), 'or', linewidth=2)

# Plot 2^{-j} as reference, these are the singular values
plt.semilogy(sp.diag(S), '-b', linewidth=2)

# Plot reference lines for machine epsilon, and sqrt(eps)
eps = np.finfo(float).eps
plt.semilogy([0,80], [eps,eps], 'g')
plt.semilogy([0,80], [np.sqrt(eps),np.sqrt(eps)], 'm')

plt.legend(['Classical', 'Modified', '$\sigma_j=2^{-j}$', '$\epsilon_{mach}$', '$\sqrt{\epsilon_{mach}}$'])

__So, what exactly is going on here?__
- $r_{jj} = \|P_j a_j\|$, so this gives us an indication of how large each projection is

- We see that the $r_{jj}$ decrease steadily, tracking (roughly) the plot of the manufactured singular values $\sigma_j$

- But, $r_{jj} \neq \sigma_j$.  These values are just roughly similar.

- Why is this? Well..
    - We can rewrite $A = U \Sigma V^T$, as 
    $$ A = 2^{-1} u_1 v^T_1 + 2^{-2} u_2 v^T_2 + \dots + 2^{-80} u_{80} v^T_{80} $$ 
    
    - Rearranging this, we can write the $j$th column of $A$ as 
    $$a_j = \left(2^{-1} \bar{v}_{j1}\right)u_1 + \left(2^{-2} \bar{v}_{j2}\right)u_2 + \dots + \left(2^{-80} \bar{v}_{j,80}\right)u_{80} $$ 
    
    - Where $ \left(2^{-1} \bar{v}_{j1}\right) $ is a scalar, and since $v$ is uniformly
    random between 0 and 1, we know that $ \bar{v}_{j1} \approx 0.1$
    
    - So, the scalar in front of each $u_i$ is on the order of $\left(0.1\, 2^{-i}\right)$.
    
    - Then during the orthogonalization for $QR$, the first column $q_1$ will be dominated by the *largest* component of $a_1$, namely $u_1$.  Thus, it will have norm on the order of $\left( 0.1\, 2^{-1} \right)$.
    
    - The next step of orthogonalization (creating $v_2$) will orthogonalize $a_2$ against $q_1$ in order to obtain $v_2$.  This will leave $v_2$ dominated by the $u_2$ term, with $\| v_2\| \sim \left( 0.1\, 2^{-2}\right)$.  
        - Remember, this $v_2$ goes on to form $q_2$
    
    - This argument repeats itself, and remembering that $r_{jj}$ equals the eventual norm of $v_j$, this explains the pattern of the diagonal entries of $R$.
    

The diagongal entries of $R$ should (roughly) decrease geometrically
- But, for classical Gram-Schmidt, these values stagnate at about $10^{-7}=\sqrt{\epsilon_{mach}}$

- Whereas for modified Gram-Schmidt, these values stagnate at about $10^{-15} = \epsilon_{mach}$

We see that modified Gram-Schmidt is able to orthogonalize up to the order of machine epsilon $\left( \epsilon_{mach} \right)$, instead of square-root of machine epsilon.

__Modified Gram-Schmit Gives Eight Orders More of Accuracy!!__

### Moving on to another experiment.  This time focusing on numerical loss of orthogonality
- Both modified and classical Gram-Schmidt may produce vectors $q_j$ that are far from orthogonal
- Although, as we've seen, modified Gram-Schmidt usually loses less precision
- This loss of orthogonality is pronounced with cond$(A)$ is *large*
- We can even see this phenomenon for very small matrices
- Consider the below $A$ and Gram-Schmidt

In [11]:
# Let's consider an ill-conditioned A, and observe loss of orthogonality
A = np.array([[0.7, 0.70711], [0.70001, 0.70711]])
from numpy.linalg import cond
np.set_printoptions(precision=5, suppress=True)
print(A)
print('---')
print(cond(A))


[[0.7     0.70711]
 [0.70001 0.70711]]
---
280016.278127884


Let's consider doing Gram-Schmidt on a machine with 5 decimal digits of precision (manually)
   - Note, all results will be rounded to 5 digits

Step $j = 1$
1. $r_{11} = 0.98996 = \| a_1\|$
     
1. $q_1 = a_1/r_{11} = 
     \begin{bmatrix} 0.70000/0.98996 \\ 0.70001/0.98996 \end{bmatrix} =
     \begin{bmatrix} 0.70710 \\ 0.70711 \end{bmatrix}$
     
Step $j=2$
1. Here, Gram-Schmidt takes $a_2$ and projects out the component of $a_2$ in the direction of $q_1$
1. $r_{12} = q^*_1 a_2 = 0.70710 \cdot 0.70711 + 0.70711 \cdot 0.70711 = 1.0000$
    - Note that the inner product of $q_1$ and $a_2$ is essentially 1.0
    - Taking that fact together with the fact that $q_1$ is norm 1, we can see that these two vectors are essentially the same (to within precision)
1. $v_2 = a_2 - r_{12} q_1 = \begin{bmatrix} 0.00001 \\ 0.00000 \end{bmatrix}$
    - This result is dominated by rounding errors.
    - Our five digits of precision indicate that $a_1$ and $a_2$ are almost
    linearly dependent
    
1. Final column of $Q$ is $q_2 = \begin{bmatrix} 1.0000 \\ 0.00000 \end{bmatrix}$

What does our modified Gram-Schmidt say $Q$ should be?

In [10]:
[Q,R] = mod_GS(A)
print(Q)

NameError: name 'mod_GS' is not defined

Whoa! That $Q$ is way different.  

But did we still lose orthogonality in our 15-16 digit precision finite arithmetic?
- Measure orthogonality loss with 
$$ \| Q^* Q - I \|_2 $$

In [ ]:
print(sla.norm(np.dot(Q.T,Q) - np.eye(2)))

Let's compare this to the library $QR$ which uses Householder (better than Gram-Schmidt for orthogonality)

In [ ]:
[Q,R] = sla.qr(A)
print(sla.norm(np.dot(Q.T,Q) - np.eye(2)))

__Thoughts__
- What if $A$ is complex valued?
    - What do we need to change above?
    - Does the order of any dot products matter?
